# Prétraitement des Données pour la Modélisation Électorale

Ce notebook a pour objectif de préparer les données brutes issues de la base de données `ELECTIONS.db`. Le processus inclut :
1. Le chargement des données agrégées.
2. Une division temporelle stricte pour créer un jeu d'entraînement et de test.
3. La création d'un pipeline de prétraitement pour standardiser les données numériques et encoder les données catégorielles.
4. La sauvegarde des données traitées et des transformateurs (pipelines) pour une utilisation future dans les notebooks d'entraînement et de prédiction.

## 1. Importation des Librairies

Nous commençons par importer toutes les librairies Python nécessaires pour la manipulation de données, la connexion à la base de données, et le machine learning.

In [38]:
# Import joblib to save and load Python objects
import joblib
import os

# Import essential libraries
import pandas as pd
import numpy as np
import sqlite3

# ML preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# ML evaluation
from sklearn.metrics import accuracy_score
print("Libs importées avec succès.")

Libs importées avec succès.


## 2. Connexion à la Base de Données

Nous établissons une connexion avec la base de données SQLite qui contient nos données électorales et socio-économiques.

In [39]:
# --- 2. CONNEXION À LA BASE DE DONNÉES ---
db_path = "../database/ELECTIONS.db"
conn = sqlite3.connect(db_path)
print(f"Connecté à la base de données : {db_path}")

Connecté à la base de données : ../database/ELECTIONS.db


## 3. Chargement et Préparation des Données Brutes

Nous exécutons une requête SQL pour agréger les données par département et par année. 

Ensuite, nous utilisons une stratégie de **forward-fill (`ffill`)**. Cela permet de propager le nom du parti gagnant des années précédentes aux années suivantes où l'information est manquante au sein d'un même département. C'est essentiel pour avoir une cible (`WINNER`) pour chaque ligne de donnée annuelle.

In [40]:
# --- 3. CHARGEMENT ET REMPLISSAGE INTELLIGENT DES DONNÉES (FORWARD-FILL) ---
query = """
SELECT
    DEPARTMENT_CODE,
    YEAR,
    WINNER,
    ROUND(AVG(POVERTY_RATE), 2) as avg_poverty_rate,
    ROUND(AVG(UNEMPLOYMENT_RATE), 2) as avg_unemployment_rate,
    ROUND(AVG(IMMIGRATION_RATE), 2) as avg_immigration_rate,
    ROUND(AVG(NUMBER_OF_VICTIMS), 0) as avg_number_of_victims
FROM ELECTIONS_ALL
GROUP BY DEPARTMENT_CODE, YEAR
ORDER BY DEPARTMENT_CODE, YEAR
"""
df_full = pd.read_sql(query, conn)

# Propager le dernier gagnant connu pour chaque département
df_full['WINNER'] = df_full.groupby('DEPARTMENT_CODE')['WINNER'].transform(lambda x: x.ffill())

# Supprimer les lignes qui restent sans gagnant (si les toutes premières années n'ont pas de valeur)
df_full.dropna(subset=['WINNER'], inplace=True)
print("Données chargées et complétées par forward-fill.")
print(f"Shape total du DataFrame après remplissage : {df_full.shape}")

Données chargées et complétées par forward-fill.
Shape total du DataFrame après remplissage : (752, 7)


## 4. Division Temporelle : Jeu d'Entraînement et Jeu de Test

Pour simuler une situation réelle où l'on prédit le futur à partir du passé, nous effectuons une **division temporelle stricte**. 
- Le jeu d'entraînement (`train_df`) contient toutes les données jusqu'à 2023.
- Le jeu de test (`test_df`) ne contient que les données de 2024.

Cela évite toute fuite de données du futur vers le passé. Nous en profitons également pour retirer la catégorie `E.GAUCHE` qui est très peu représentée et pourrait nuire à la qualité du modèle.

In [41]:
# --- 4. DIVISION TEMPORELLE STRICTE (TRAIN < 2024, TEST = 2024) ---
train_df = df_full[df_full['YEAR'] < 2024].copy()
test_df = df_full[df_full['YEAR'] == 2024].copy()

# Par sécurité, on retire 'E.GAUCHE' qui a très peu d'échantillons et peut nuire à l'apprentissage
train_df = train_df[train_df['WINNER'] != 'E.GAUCHE']

print(f"\nTaille du jeu d'entraînement (2017-2023) : {train_df.shape}")
print(f"Taille du jeu de test (2024) : {test_df.shape}")


Taille du jeu d'entraînement (2017-2023) : (652, 7)
Taille du jeu de test (2024) : (94, 7)


## 5. Définition des Features (X) et de la Cible (y)

Nous séparons nos jeux de données en variables explicatives (X) et en variable cible (y). Les colonnes `WINNER` et `YEAR` sont exclues des features.

In [42]:
# --- 5. DÉFINITION DES FEATURES (X) ET DE LA CIBLE (y) ---
# Les features sont toutes les colonnes sauf 'WINNER' et 'YEAR'
X_train = train_df.drop(columns=['WINNER', 'YEAR'])
y_train = train_df['WINNER']
X_test = test_df.drop(columns=['WINNER', 'YEAR'])
y_test = test_df['WINNER']

print(f"\nJeux de données X_train/y_train et X_test/y_test créés.")


Jeux de données X_train/y_train et X_test/y_test créés.


## 6. Création du Pipeline de Pré-traitement

Nous construisons un `ColumnTransformer` pour traiter différemment les types de colonnes :
- **Variables Numériques** (`avg_poverty_rate`, etc.) : Seront standardisées (mises à l'échelle) avec `StandardScaler`.
- **Variables Catégorielles** (`DEPARTMENT_CODE`) : Seront transformées en variables binaires avec `OneHotEncoder`.

Parallèlement, nous créons un `LabelEncoder` pour convertir les noms des partis (texte) en chiffres.

In [43]:
# --- 6. CRÉATION DU PIPELINE DE PRÉTRAITEMENT ---
# Identification des colonnes
categorical_features = ['DEPARTMENT_CODE']
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()

# Création du preprocessor pour les features (X)
preprocessor_X = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Création de l'encodeur pour la cible (y)
label_encoder_y = LabelEncoder()
print("\nPipeline de pré-traitement et encodeur de label créés.")


Pipeline de pré-traitement et encodeur de label créés.


## 7. Application des Transformations sur les Données

C'est une étape cruciale. On **apprend** les paramètres de transformation (la moyenne pour la standardisation, la liste des catégories pour l'encodage, etc.) **uniquement sur le jeu d'entraînement** (avec `.fit_transform()`).

Ensuite, on applique cette **même transformation**, sans ré-apprendre, sur le jeu de test (avec `.transform()`). Cela garantit que notre modèle ne "voit" aucune information du jeu de test pendant sa phase de préparation.

In [44]:
# --- 7. APPLICATION DES TRANSFORMATIONS ---
# On "fit" (apprend) sur le jeu d'entraînement et on transforme les deux jeux
X_train_processed = preprocessor_X.fit_transform(X_train)
X_test_processed = preprocessor_X.transform(X_test)

y_train_encoded = label_encoder_y.fit_transform(y_train)
y_test_encoded = label_encoder_y.transform(y_test)
print("\nTransformations appliquées aux données d'entraînement et de test.")
print(f"Shape de X_train_processed: {X_train_processed.shape}")
print(f"Shape de X_test_processed: {X_test_processed.shape}")


Transformations appliquées aux données d'entraînement et de test.
Shape de X_train_processed: (652, 98)
Shape de X_test_processed: (94, 98)


## 8. Sauvegarde des Données Traitées et des Transformateurs

Maintenant que nos données sont propres et prêtes pour la modélisation, nous les sauvegardons. Nous sauvegardons également les transformateurs pour pouvoir appliquer les mêmes transformations sur de nouvelles données à l'avenir.

### 8.1. Reconstruction des DataFrames

La sortie du pipeline est un tableau NumPy. Nous le convertissons en DataFrame pandas en récupérant les noms des colonnes pour une meilleure lisibilité et pour la sauvegarde en SQL.

In [45]:
# --- 8. PRÉPARATION DES DONNÉES FINALES POUR LA SAUVEGARDE ---
# Récupération des noms des colonnes après one-hot encoding
try:
    feature_names = preprocessor_X.get_feature_names_out()
except AttributeError: # Fallback pour les anciennes versions de scikit-learn
    ohe_feature_names = preprocessor_X.named_transformers_['cat']['onehot'].get_feature_names(input_features=categorical_features)
    feature_names = numerical_features + list(ohe_feature_names)

In [46]:
# Création des DataFrames finaux
X_train_processed_df = pd.DataFrame(X_train_processed.toarray() if hasattr(X_train_processed, "toarray") else X_train_processed, columns=feature_names)
X_test_processed_df = pd.DataFrame(X_test_processed.toarray() if hasattr(X_test_processed, "toarray") else X_test_processed, columns=feature_names)

# Ajout de la cible encodée pour la sauvegarde
train_to_db = X_train_processed_df.copy()
train_to_db['WINNER_encoded'] = y_train_encoded

test_to_db = X_test_processed_df.copy()
test_to_db['WINNER_encoded'] = y_test_encoded
print("\nDataFrames finaux prêts pour la sauvegarde.")


DataFrames finaux prêts pour la sauvegarde.


### 8.2. Sauvegarde dans la base de données

In [47]:
# --- 9. SAUVEGARDE DANS LA BASE DE DONNÉES ET DES TRANSFORMATEURS ---
# Sauvegarde des DataFrames dans SQLite
train_to_db.to_sql('PROCESSED_TRAIN_DATA', conn, if_exists='replace', index=False)
test_to_db.to_sql('PROCESSED_TEST_DATA', conn, if_exists='replace', index=False)
print(f"\nDonnées sauvegardées dans les tables 'PROCESSED_TRAIN_DATA' ({train_to_db.shape}) et 'PROCESSED_TEST_DATA' ({test_to_db.shape}).")


Données sauvegardées dans les tables 'PROCESSED_TRAIN_DATA' ((652, 99)) et 'PROCESSED_TEST_DATA' ((94, 99)).


### 8.3. Sauvegarde des transformateurs

C'est l'étape la plus critique pour l'inférence future. En sauvegardant les objets `preprocessor_X` et `label_encoder_y` avec `joblib`, nous pourrons plus tard charger ces objets pour appliquer **exactement les mêmes transformations** sur de nouvelles données avant de faire une prédiction.

In [48]:
# Sauvegarde des transformateurs en fichiers .joblib
db_dir = os.path.dirname(db_path)
preprocessor_path = os.path.join(db_dir, 'preprocessor_X.joblib')
label_encoder_path = os.path.join(db_dir, 'label_encoder_y.joblib')

joblib.dump(preprocessor_X, preprocessor_path)
joblib.dump(label_encoder_y, label_encoder_path)
print(f"Préprocesseur sauvegardé dans : {preprocessor_path}")
print(f"Encodeur de label sauvegardé dans : {label_encoder_path}")

Préprocesseur sauvegardé dans : ../database/preprocessor_X.joblib
Encodeur de label sauvegardé dans : ../database/label_encoder_y.joblib


## 9. Fermeture de la Connexion

Finalement, nous fermons la connexion à la base de données pour libérer les ressources.

In [49]:
# --- 10. FERMETURE DE LA CONNEXION ---
conn.close()
print("\nConnexion à la base de données fermée. Script terminé.")


Connexion à la base de données fermée. Script terminé.
